# Water flow in soil - Numerical solutions in DuMu${}^x$

_by Daniel Leitner, Timo Koch, Andrea Schnepf_

In the following we will give some hints how to solve Benchmarks 1-4, using [DuMu${}^x$](www.dumux.org). Scripts for the following benchmarking examples are available on [Github](https://github.com/Plant-Root-Soil-Interactions-Modelling/dumux-rosi.git), in the folder dumux-rosi/rosi_benchmarking/soil/ 

For 1D run /python/dumux_1d\*.py, for 3D /python/dumux_3d\*.py, for unstructured grids /python/dumux_ug\*.py.

To run the examples you will need: dune (releases/2.6), dune-foamgrid (releases/2.6), dumux (releases/3.0), CRootBox (master)


## Theory

We base our Dumux implementation on the [Richards Problem](http://www.dumux.uni-stuttgart.de/doxygen-stable/html-2.12/a01911.php) which states the Richards equation in the following form:
\begin{eqnarray} 
\frac{\partial}{\partial t} \left( \phi S \rho \right) - \frac{\partial}{\partial z}  \left[\rho \frac{\kappa}{\mu} K_s \left(\frac{\partial \psi}{\partial z} - \rho g \right) \right] = 0,
\end{eqnarray}
where $\rho$ is the fluid density, $S$ is the fluid saturation, $\phi$ is the porosity, $\kappa$ is the relative permeability, $\mu$ the dynamic viscosity, $K_s$ the intrinsic permeability, $\psi$ the fluid pressure and $g$ the gravitational acceleration. The quantities $S$ and $\psi$ are related by the water retention curve, i.e. $S=f^{-1}(\psi_{ref}-\psi) $

Introducing the hydraulic conductivity $K_c = K \ \rho \ g / \mu$, with $K = \kappa \ K_s$, and the water content $\theta = \phi \ S$ the equation takes the form 
\begin{eqnarray} 
\frac{\partial}{\partial t} \left( \rho \theta \right) = \frac{\partial}{\partial z}  \left[ K_c \frac{1}{g} \left(\frac{\partial \psi}{\partial z} - \rho g \right) \right].
\end{eqnarray}

Using the pressure head $h = (\psi-\psi_{ref})/(\rho \ g)$ leads to
\begin{eqnarray} 
\frac{\partial}{\partial t} \left( \rho \theta \right) = \frac{\partial}{\partial z}  \left[ \rho K_c \left(\frac{\partial h}{\partial z} - 1 \right) \right],
\end{eqnarray}
and dividing the equation by $\rho$ leads to the Richards equation, as presented in [Water flow in soil](Soil water flow.ipynb) (the fluid density $\rho$ must be constant in space and time). 

## Compile program

We compile the program by calling 

*make richards1d* (for 1D)

*make richards3d* (for 3D)

*make richardsUG* (for unstructured grids)

in the folder dumux-rosi/build-cmake/rosi_benchmarking/soil.

## Run benchmarks
All benchmarks can be simulated by just changing the Dumux .input files. The specific benchmarks can be started by Python scripts located in the folder dumux-rosi/rosi_benchmarking/soil/python.

## Input files

The input files for benchmarks 1-4 are located in /soil/benchmarks_1d/ (for 1D), /soil/benchmarks_3d/ (for 3D), and /soil/benchmarks_ug/ (for unstructured grids). The following parameters are used:

**[Problem]**

*Name*, name of the simulation

**[TimeLoop]**

*TEnd*, simulation time [s]. If the value is zero a steady state problem is calculated

*DtInitial*, initial time step [s]

*MaxTimeStepSize*, maximal time step size [s]

*CheckTimes*, optionally times [s] can be defined, when the .vtp output is written. Per default a file at start and at the final time is written.

**[Soil.Grid]**

*UpperRight*, in 1d actually just the upper coordinate (per default = 0, in all benchmarks) [m]

*LowerLeft*, lower coordinate [m]

*Cells*, number of cells (degrees of freedom *dof*-1 for discretization with box method) 

or

*File*, name of the 1d dune grid file (.dgf)

*layerIdx*, optionally defines where to find soil layer numbers in grid file (default = 1).

**[Soil.BC.Top]**

*Type*, type of the top boundary condition (1 = dirichlet, 2 = neumann, 4 = atmospheric, see [Climate])

*Value*, [cm] pressure head for dirchlet (type 1), and [cm/day] for neumann or atmospheric (type 2 & 4) 

**[Soil.BC.Bot]**

*Type*, type of the top boundary condition (1 = dirichlet, 2 = neumann, 5 = free drainage)

*Value*, [cm] pressure head for dirchlet (type 1), and [cm/day] for neumann (type 2). 

**[Soil.VanGenuchten]**

The set of Van Genuchten parameters: *Qr* [1], *Qs* [1], *Alpha* [1/cm], *N* [1], and *Ks* [cm/day]. If there are multiple soil layers, each parameter is given by multiple values (i.e. one value per soil layer). 

**[Soil.IC]**

*P*, *Z*

Initial soil conditions are given by an input file function [cm] ([m]), cm pressure head at a depth (in meter), see InputFileFunction (in dumux-rosi/dumux/io/inputfilefunction.h). 

If *P* is a single value constant pressure is assumed. If there are multiple values, and additionally, *Z* values are defined, tabular linear interpolated data is assumed. If there are multiple values, but no *Z* values, the pressure is assumed to be given constant per soil layer.

Optionally, ascending values are given for linear interpolation, e.g. *Z* = [-2, -1, -1, 0], *P*= [-400,-400,-200,0], would be -400 cm between 2 and 1 meter depth, and linear increasing from -200 cm to full saturation between 1 meter depth and the surface. 

The tabulated data can also be given in .csv file, stating a *File* instead of *P* and *Z*.

**[Soil.Layer]**

*Number*, *Z*  

The soil layer number is given by an input file function [1] ([m]), i.e. layer number at depth (in meter), see InputFileFunction (in dumux-rosi/dumux/io/inputfilefunction.h). 

The layer number might be given within the grid file as parameter with index *layerIdx*. In this case we do not need to set the [Soil.Layer] tag, but the presence of layer data is indicated by multiple VanGenuchten parameters.

**[Climate]**

*Precipitation*, *Time* 

Climate data are given by an input file function [kg/day] ([day]), i.e. water loss rate (kg/day) at a time (in days) see InputFileFunction (in dumux-rosi/dumux/io/inputfilefunction.h).

Negative values mean precipitation, positive evaporation.

The atmospheric boundary condition is implemented with surface run-off, and the critical evaporation pressure is -10000 [cm] per default, or can be be given by *CriticalPressure*. 



## Remarks

The Benchmark simulations can be executed by running the corresponding Python scripts located in the python/ folders. The corresponding .input files are located in the folders benchmarks_1d/, benchmarks_3d/, benchmarks_ug/, for 1d, 3d, and unstructured 3d grids.

### Benchmark 1 (static flow)

Benchmark 1 has multiple (two) sets of Van Genuchten parameters. The soil layer is given by linear interpolation based on the z-coordinate within the soil.

When calculating steady state it is sufficient to set *TimeLoop.TEnd* to zero. But sometimes the solution is not found. In this case either choose better initial conditions (which is used as initial guess), or set *TimeLoop.TEnd* to a very large value. 

In 1D we chose a large value, in 3D we picked a good initial guess and calculated the steady state example. 

For unstructured grids we only calculate steady state for subplot (c), but choose a large value for (a) & (b). Even when calculating in parallel the unstructured solution takes some time.

### Benchmark 2 (static evaporation)

Initial condition is given by linear interpolation. All benchmarks are calculated as steady state problems, i.e. TimeLoop.TEnd = 0.

### Benchmark M2.1(dynamic infiltration)

The output times are defined with TimeLoop.CheckTimes.

Note that for top boundary condition we use atmospheric, and give the precipitation in the [Climate] tag. In this way the boundary condition ensures that the influx cannot be higher than the maximal infiltration. 

For 3D examples the number of CPU cores that are used can be defined in the Python script.


### Benchmark M2.2 (dynamic evaporation) 

Again, we choose atmospheric boundary conditions as top boundary condition.

When we choose atomsopheric boundary conditions additionally a .csv file ist written, containing two values per line: simulation time [s] and actual precipitation [kg/(m²\*s)]. We read this file for visualization of the results.

Note that in 3D results do not get better if we only refine the $z$-axis.


## Implementation

In the following we describe the code, that is used to solve Benchmarks 1-4.

The type of model is chosen by including the right header file in the problem file, with ```#include <dumux/porousmediumflow/richards/model.hh>```. Additionally, the spatial parameter class is derived from a suitable base class, i.e. ```FVSpatialParams```.


### main file: richards.cc 

Compilation with make richards1d, richards3d, and richardsUG compiles this file with different grid types, defined as variable GRIDTYPE in the CMakeLists.txt file. The main file is a normal Dumux 3.0 main file with the following extensions:

1. The TimeLoop.CheckTimes are set in L108-124.

2. We check if the example is steady state or dynamic and calculate it accordingly. 

3. We only write vtk files at the check times L165-167.



### problem file: richardsproblem.hh

A Dumux Problem class describes initial conditions, boundary conditions, and sources and sinks of the problem. Sources and sinks are not needed for the benchmarks.

First, the constructor reads all relevant parameters from the input file, in our case: Soil.BC.Top.Type, Soil.BC.Top.Value, Soil.BC.Bot.Type, Soil.BC.Bot.Value, Climate.Precipitation, Climate.Time, Soil.IC.P, Soil.IC.Z.

*Definition of boundary conditions:*

We have to choose the type of Dumux boundary condition in the member function boundaryTypesAtPos(...), and set it to either Neumann or Dirichlet.

Dirichlet boundary conditions can not be solution dependent. They are used for constant pressure top or bot (see dirichletAtPos(...)).  

Neumann boundary conditions can be solution dependent, they can be used to impose Dirichlet boundary conditions (by calculating the right non-linear flux). Neumann is used for constant flux, atmospheric, and free drainage (see neumann(...)). For atmospheric top boundary condition additionally a .csv file is written, containing time, and actual precipitation or evaporation. 

*Definition of initial condition* 

Performed with the method initial(...). You can check in first vtk, that is written, if initial data are correct.
Initial conditions can be constant, or given by a table. This is performed by the helper class InputFileFunction in dumux-rosi/dumux/io/inputfilefunction.hh.


### spatial parameters: richardsparams.hh

This class manages the spatial parameters. 

The constructor reads all relevant parameters form the input file, these are the sets of Van Genuchten parameters for each layer (only one in homogeneous soil), and the layer data.

porosityAtPos(...) must be overwritten, but model solution is independent of it (no porosity in the richards equation).

permeability(...)  defines the (intrinsic) permeability $K$ [m^2] and must be overwritten. Permeability is calculated from the hydraulic conductivity $K_c$ from the Van Genuchten parameters [m/s]:
\begin{eqnarray} 
K = K_c \frac{\mu}{\rho g},
\end{eqnarray}
where $\mu$ is dynamic viscosity [Pa s] = [kg m$^{−1}$ s$^{-1}$] , $\rho$ is the fluid density [kg m$^{-3}$], and $g$ is the gravitational acceleration [m s$^{-2}$], see Eqn (2).

materialLawParams(...) method returns the Van Genuchten parameter for an element of type ```
EffToAbsLaw<RegularizedVanGenuchten<Scalar>>```
